In [ ]:
import json
import requests
import pandas as pd
#from config import gkey
from pprint import pprint
import matplotlib.pyplot as plt
from scipy.stats import linregress
import numpy as np

In [ ]:
#Read the LA restaurants csv file 
file = "Covid_LA.csv"
la_covid = pd.read_csv(file)
la_covid.head()


In [ ]:
#number of full-service restaurants in each city
data = "https://data.lacity.org/resource/6rrh-rzua.json?"
field = "Full-service restaurants"


y= 0
for x in la_covid['City']:
    target = data + f"city={x}" + f"&primary_naics_description={field}" + "&$limit=5000"  
    response = requests.get(target).json()
    restaurants = len(response)
    la_covid.loc[y, '# of restaurants'] = restaurants
    y = y + 1
    print(f"Total of restaurants in city {x} is {restaurants}")
   

In [ ]:
#city name to UpperCase
la_covid['City'] = la_covid['City'].str.upper()
la_covid.head()

In [ ]:
la_covid.head(5)

In [ ]:
#clean empty spaces 
indexNames = la_covid[(la_covid['Number_of_restaurants'] == 0)].index
la_covid.drop(indexNames, inplace=True)
la_covid.head()

In [ ]:
#cleanup dataframe 
la_covid['Cases'] = la_covid['Cases'].astype(float)
la_covid['Case_rate'] = la_covid['Case_rate'].astype(float)
la_covid['Number_of_restaurans'] = la_covid['Number_of_restaurants'].astype(float)
la_covid['Population'] = round(la_covid['Cases']/la_covid['Case_rate']*100000)
#la_covid['Population'] = la_covid['Population'].astype(float)
la_covid['Restaurants_per1k'] = (la_covid['Number_of_restaurants']/la_covid['Population']*1000)
la_covid.head()

In [ ]:
#plot 
case_rate = la_covid ['Case_rate']/1000
restaurants = la_covid ['Restaurants_per1k']


(slope, intercept, rvalue, pvalue, stderr) = linregress(restaurants, case_rate)
regress_values = restaurants * slope + intercept
line_eq = "y = " + str(round(slope,3)) + "x + " + str(round(intercept,2))


plt.scatter(restaurants, case_rate)
plt.plot(restaurants, regress_values, "r-")
plt.annotate(line_eq,(8,2),fontsize=15,color="black")
plt.xlabel('Restaurants per 1k capita')
plt.ylabel('Cumulative Cases (%)')
plt.title('Cases vs Restaurants in LA')
#plt.xlim(0, 0.04)
plt.savefig('Cases_vs_Restaurants_LA_full')
plt.show()
print(f"The r value is {round(rvalue,2)}")
print(f"The p value is {round(pvalue, 2)}")

In [ ]:
#open and write to new csv file 
la_covid.to_csv('Cases_vs_La_Restaurants.csv')


In [ ]:

# Calculate the quantitatively determine if there are any potential outliers. \n",
quartiles = la_covid['Restaurants_per1k'].quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq
lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)

la_covid_adj = la_covid.loc[(la_covid['Restaurants_per1k'] < upper_bound) & (la_covid['Restaurants_per1k'] > lower_bound)]
print(len(la_covid))
print(len(la_covid_adj))


In [ ]:
#plot with outliers removed
restaurants_adj = la_covid_adj['Restaurants_per1k']
cases_adj = la_covid_adj['Case_rate']/1000

(slope, intercept, rvalue, pvalue, stderr) = linregress(restaurants_adj, cases_adj)
regress_values = restaurants_adj * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

plt.scatter(restaurants_adj, cases_adj)
plt.plot(restaurants_adj,regress_values,"r-")
plt.annotate(line_eq,(1.25,4),fontsize=15,color="black")
plt.annotate(f"The r value is {round(rvalue,2)}", (1.25,3.5))
plt.xlabel('Restaurants per 1k capita')
plt.ylabel('Cumulative Cases (%)')
plt.title('Cases vs restaurants in LA')
#plt.xlim(0,4)
plt.savefig('Cases_vs_restaurants_LA_Adj')
plt.show()
print(f"The r value is {round(rvalue,2)}")
print(f"The p value is {round(pvalue, 2)}")

In [ ]:
#find the low for the restaurant 
lacovid_low = la_covid.loc[(la_covid['Restaurants_per1k']>0.023) & (la_covid['Restaurants_per1k']<0.025)]
lacovid_low_covid= lacovid_low['Cases']
lacovid_low_covid


In [ ]:
#show the data Getting an error here 
data = "https://data.lacity.org/resource/6rrh-rzua.json?"
target  = data + f"&Cases =1544.0"
response = requests.get(target).json()
pprint (response)

In [ ]:
timeline_low = pd.DataFrame()
timeline_low['Week']=[]
timeline_low['case_rate_weekly']=[]

In [ ]:
for x in range(len(response)):
    try: 
        week = response[x]['week_number']
        case_rate =response[x]['case_rate_weekly']
        timeline_low.loc[x,'Week']= week
        timeline_low.loc[x,'case_rate_weekly']= case_rate
    except: KeyError
        
timeline_low1 = timeline_low.sort_values(by=['Week'])
timeline_low1


In [ ]:
#find the high for the restaurant 
lacovid_high = la_covid.loc[(la_covid['Restaurants_per1k']>0.005) & (la_covid['Restaurants_per1k']<0.005)]
lacovid_high_covid= lacovid_high['Cases']
lacovid_high_covid

In [ ]:
#print out the data Getting an error 
data = "https://data.lacity.org/resource/6rrh-rzua.json?"
target  = data + f"&Cases ="
response = requests.get(target).json()
pprint (response)

In [ ]:
timeline_high = pd.DataFrame()
timeline_high['Week']=[]
timeline_high['case_rate_weekly']=[]

In [ ]:
for x in range(len(response)):
    try: 
        week = response[x]['week_number']
        case_rate =response[x]['case_rate_weekly']
        timeline_high.loc[x,'Week']= week
        timeline_high.loc[x,'case_rate_weekly']= case_rate
    except: KeyError
        
timeline_high1 = timeline_high.sort_values(by=['Week'])
timeline_high1

In [ ]:
#create the timeline 
timeline_high1 = timeline_high.sort_values(by=['Week'])
timeline_high1

In [ ]:
#plot time by week
x1 = timeline_low1['Week'].astype(float)
x2 = timeline_high1['Week'].astype(float)
y1 = timeline_low1['case_rate_weekly'].astype(float)
y2 = timeline_high1['case_rate_weekly'].astype(float)
plt.plot(x1,y1,)
plt.plot(x2,y2,)
plt.legend(['Many Restaurants', 'Few Restaurants'],loc='best')
plt.title('Covid case timeline')
plt.xlabel('Week')
plt.ylabel('Case rate (/100k)')
plt.savefig('Covid_timeline_Low_vs_High_Restaurant_density')
plt.show()